In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上执行端到端的机器学习：MLOps阶段6：serving：开始重新导入AutoML表格模型

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_exported_deploy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_exported_deploy.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_exported_deploy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在 Google Cloud 上生产环境中使用 Vertex AI 进行端到端的 MLOps。本教程涵盖了第 2 阶段：实验：使用 AutoML Training 入门。

### 目标

在本教程中，您将学习如何使用 `AutoML Tabular` 将导出的模型构件重新导入为 `Model` 资源。例如，如果要将导出的模型移动到其他项目中，这将非常有用。

本教程使用以下 Google Cloud ML 服务：

- `AutoML Tabular`
- `Vertex AI Model` 资源
- `Vertex AI Prediction`

执行的步骤包括：

- 将预训练的 AutoML tabular 导出的模型构件导入为 `Model` 资源。
- 创建一个 `Endpoint` 资源。
- 将 `Model` 资源部署到 `Endpoint` 资源。
- 进行预测。

数据集

本教程使用了一个预训练的自动机器学习表格模型，其中包含导出的模型工件。

用于预训练模型的表格数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您所使用的数据集版本仅使用年、月和日字段来预测平均每日温度值（mean_temp）。

*注意：*这个导出模型的版本包含自定义操作，并且需要模型服务器：us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server.

费用
本教程使用谷歌云的可计费组件:

- Vertex AI
- 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装所需的软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install --upgrade google-cloud-storage {USER_FLAG} -q

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始前

### 设置您的 Google Cloud 项目

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。创建账户时，您将获得$300的免费信用用于支付计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI，Compute Engine 和 Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保
Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于此笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，可能会使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Vertex AI Workbench Notebooks**，您的环境已经通过验证。请跳过此步骤。

**如果您正在使用 Colab**，请运行下面的单元格，并按提示进行身份验证，通过 oAuth。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

1. **点击创建服务帐号**。

2. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

3. 在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入 "Vertex AI"，选择**Vertex AI管理员**。在过滤框中输入 "Storage Object Admin"，选择**存储对象管理员**。

4. 点击创建。包含您的密钥的 JSON 文件将下载到您的本地环境中。

5. 在下面的单元格中，输入您的服务帐号密钥路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

当您为 Python 初始化 Vertex SDK 时，您需要指定一个云存储暂存桶。该暂存桶是您的数据集和模型资源在会话之间保留的地方。

请在下面设置您的云存储存储桶的名称。存储桶的名称必须在所有 Google 云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的桶不存在时才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 为 Python 初始化 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 个CPU核心

*注意：以下内容不支持训练:*

 - `standard`: 2个 vCPUs
 - `highcpu`: 2、4 和 8 个 vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 预训练 `AutoML Tabular` 导出模型的位置

现在将变量 `MODEL_PACKAGE` 设置为云存储中导出模型的位置。

In [ ]:
MODEL_PACKAGE = "gs://cloud-samples-data/vertex-ai/tabular-workflows/models/custom_op"

快速查看您的模型包。

接下来，请查看导出的AutoML表格模型的模型包内容。

In [ ]:
! gsutil ls {MODEL_PACKAGE}

## AutoML 表格模型

AutoML 可以训练以下类型的表格模型：

- 分类
- 回归
- 预测

模型可以被训练用于自动部署到云端，或者导出为手动部署到云端。在本教程中，您将使用预训练的导出的 AutoML 表格模型。

了解更多有关[AutoML 模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

了解更多有关[导出 AutoML 表格模型](https://cloud.google.com/vertex-ai/docs/export/export-model-tabular)

### 设置模型服务器

接下来，您将为模型服务器设置预构建的容器。该容器将是`us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server`的一个版本。如果模型包含一个`environment.json`文件，则使用由键`container_uri`指定的容器版本；否则使用`us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:latest`。

In [ ]:
import json

output = !gsutil cat {MODEL_PACKAGE}/environment.json

MODEL_SERVER = json.loads(output[0])["container_uri"]

### 将预训练导出的`AutoML Tabular`模型包上传到`Vertex AI Model`资源

接下来，您可以使用以下参数将预训练导出的`AutoML Tabular`模型的模型工件上传到`Vertex AI Model`资源中，利用`Model.upload()`方法：

- `display_name`：`Model`资源的可读名称。
- `artifact_uri`：模型包的云存储位置。
- `serving_container_image_uri`：用于serving的容器映像。
- `serving_container_ports`：提供服务的端口。

*注意：*在将模型工件上传到`Vertex Model`资源时，您需要指定相应的部署容器映像。

In [ ]:
model = aiplatform.Model.upload(
    display_name="gsod_" + TIMESTAMP,
    artifact_uri=MODEL_PACKAGE,
    serving_container_image_uri=MODEL_SERVER,
    serving_container_ports=[8080],
)

## 创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。最低要求是为端点指定显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您在使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的易读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）用户定义的元数据，以键/值对形式保存到`Endpoint`中。

此方法将返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="gsod_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点。每个部署的 `Vertex AI Model` 资源都会有自己的用于提供二进制文件的部署容器。

*注意:* 在上传模型工件到 `Vertex AI Model` 资源的上一个步骤中，您为导出的 AutoML 表格模型指定了部署容器。

要进行部署，您需要指定以下附加配置设置：

- 机器类型。
- (如果有的话) GPU 的类型和数量。
- 静态、手动或自动扩展 VM 实例。

在本示例中，您可以使用最少数量的指定参数来部署模型，如下：

- `model`: `Model` 资源。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `machine_type`: 每个 VM 实例的机器类型。

由于资源的规划需求，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="gsod_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(response)

进行预测

最后，您可以使用`endpoint（）`方法进行在线预测，参数如下：

- `instances`：要预测的实例。

以下是一个预测请求示例：

    [ INSTANCE_1，INSTANCE_2，... ]
    
    INSTANCE：{“column_1”：value，“column_2”：value，...}

In [ ]:
INSTANCES = [{"year": "2020", "month": "1", "day": "23"}]

prediction = endpoint.predict(instances=INSTANCES)
print(prediction)

删除该端点

方法'delete()'将删除该端点。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

删除该模型

方法'delete()'将删除该模型。

In [ ]:
model.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI